Read CSV
Generate Table with headings
Generate y_train data
    Read Steering Angle
Generate X_train data
    Read images (center, left, right) using filename
    Crop
    Augment
        Flip,resize - append(y_train, offset)
        Noise, resize - append(y_train, offset)
        blur, resize - append(y_train, offset)

Neural Network Implmentation
training, validation, testing split
Generator Function

Neural Network Training
Save Model

    

### Read CSV

In [9]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
csv_headers = ["center", "left", "right", "steering", "throttle", "brake", "speed"]

data = pd.read_csv('CarSim_data/driving_log.csv', names=csv_headers)

In [ ]:
data.head()

### Read Image, Crop and Resize

In [ ]:
for i in data['center']:
    print(i)

### Features

In [ ]:
img_path = data['center'][1]
#print(img_path)
num_samples = len(data)
i=0
for row in data['center']:
#     print(i)
    I = cv2.imread(img_path)
    b,g,r = cv2.split(I)
    I = cv2.merge([r,g,b])
    #cv2.imwrite('I.jpg',I)
    I = I[50:,:]
    I = cv2.resize(I,None,fx=0.25,fy=0.25,interpolation=cv2.INTER_AREA)
    plt.imshow(I)
    
    # https://carnd-forums.udacity.com/questions/32716893/
    # does-anyone-know-how-to-make-multiple-images-into-4d-arrays-like-pickled-data#
    #arr = np.zeros((num_examples, width, height, channels))

    X_train = np.zeros((num_samples, I.shape[0], I.shape[1], I.shape[2]))
    X_train[i, :, :, :] = I
    i=i+1

In [ ]:
print(type(I))

### Labels

In [ ]:
# Convert a pandas dataframe (df) to a numpy ndarray
y_train = data['steering'].values
print(y_train.shape)

### Create Features and Labels

### Preprocess the Data
1.) Shuffle the data

2.) Normalize the features using Min-Max scaling between -0.5 and 0.5

3.) Steering Angle: -25 to 25


In [ ]:
data.shape
len(data)

In [ ]:
I.shape[2]

In [ ]:
# https://carnd-forums.udacity.com/questions/32716893/
# does-anyone-know-how-to-make-multiple-images-into-4d-arrays-like-pickled-data#
#arr = np.zeros((num_examples, width, height, channels))

features = np.zeros((len(data), I.shape[1], I.shape[0], I.shape[2]))
features[i, :, :, :] = I


In [ ]:
arr.shape

### Generator

In [ ]:
import sklearn
import pandas
from sklearn.utils import shuffle

In [ ]:
shuffle

In [ ]:
def myGenerator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # loop forever so generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './IMG'+batch_sample[0].split('/')[-1]
                center_image = cv2.imreade(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

In [ ]:
# compile and train the model using generator function
train_generator = myGenerator(train_samples, batch_size=32)
validation_generator = myGenerator(validation_samples, batch_size=32)

## Keras Model

In [1]:
from keras.models import Sequential
from keras.layers import Activation, Convolution2D, Dense, Dropout, Flatten, Lambda, MaxPooling2D


Using TensorFlow backend.


In [2]:
ch, row, col = 3,64,64

In [12]:
model = Sequential()

model.add(Lambda(lambda x: x/127.5-1., input_shape=(col, row, ch), output_shape=(col, row, ch)))


model.add(Convolution2D(24,5,5, init='glorot_uniform', border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(36,5,5, init='glorot_uniform', border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(48,5,5, init='glorot_uniform', border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(64,3,3, init='glorot_uniform', border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Convolution2D(64,3,3, init='glorot_uniform', border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1164))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(1))